#사전 작업

In [ ]:
!pip install shap

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#데이터 전처리


In [84]:
#import 및 파일 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
import re
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_squared_log_error

# 파일 경로 설정
file_path = "/content/drive/MyDrive/Colab Notebooks/01 DATA/accidentInfoList.CSV"

# 데이터 불러오기
df = pd.read_csv(file_path, encoding='cp949')



In [85]:
# 숫자 추출 함수 생성
def extract_numbers_from_age(age):
    numbers = re.findall(r'\d+', str(age))
    if numbers:
        return int(numbers[0])
    else:
        return np.nan

# '가해운전자 연령' 및 '피해운전자 연령'에서 숫자만 추출
df['가해운전자 연령'] = df['가해운전자 연령'].apply(extract_numbers_from_age)
df['피해운전자 연령'] = df['피해운전자 연령'].apply(extract_numbers_from_age)

# 결측치 처리
mean_age_driver = round(df['가해운전자 연령'].mean(), 1)
mean_age_victim = round(df['피해운전자 연령'].mean(), 1)
df['가해운전자 연령'].fillna(mean_age_driver, inplace=True)
df['피해운전자 연령'].fillna(mean_age_victim, inplace=True)
df['피해운전자 차종'].fillna('기타불명', inplace=True)
df['피해운전자 성별'].fillna('기타불명', inplace=True)

In [86]:
# 피처 엔지니어링

# 'ECLO' 피처 생성
df['ECLO'] = df['사망자수'] * 10 + df['중상자수'] * 5 + df['경상자수'] * 3 + df['부상신고자수'] * 1
df.drop(columns=['사망자수', '경상자수', '중상자수', '부상신고자수'], inplace=True)

# '사고일시'에서 시간 정보 추출
df['사고일시'] = pd.to_datetime(df['사고일시'], format='%Y년 %m월 %d일 %H시')
df['사고발생시간'] = df['사고일시'].dt.hour
df.drop(columns=['사고일시'], inplace=True)

# 피처 제거
df.drop(columns=['사고번호', '시군구','사고내용'], inplace=True)

In [87]:
#범주형 데이터 처리

# 요일을 평일/주말로 변환
def categorize_weekday(weekday):
    if weekday in ['월요일', '화요일', '수요일', '목요일', '금요일']:
        return '평일'
    else:
        return '주말'

df['평일/주말'] = df['요일'].apply(categorize_weekday)
df.drop(columns=['요일'], inplace=True)

# 사고발생시간을 주간/야간으로 변환
def recategorize_time_of_day(hour):
    if 21 <= hour <= 24 or 0 <= hour < 6:
        return '야간'
    else:
        return '주간'

df['시간대'] = df['사고발생시간'].apply(recategorize_time_of_day)
df.drop(columns=['사고발생시간'], inplace=True)

# 연령대로 범주화
def categorize_age(age):
    if age <= 19:
        return '미성년층'
    elif 20 <= age <= 45:
        return '청년층'
    elif 46 <= age <= 60:
        return '중장년층'
    else:
        return '노년층'

df['가해운전자 연령대'] = df['가해운전자 연령'].apply(categorize_age)
df.drop(columns=['가해운전자 연령'], inplace=True)
df['피해운전자 연령대'] = df['피해운전자 연령'].apply(categorize_age)
df.drop(columns=['피해운전자 연령'], inplace=True)

# 노면상태 범주화
dangerous_conditions = ['서리/결빙', '적설', '침수', '해빙', '기타']
df['노면상태_위험도'] = df['노면상태'].apply(lambda x: '위험(적설, 침수)' if x in dangerous_conditions else x)
df.drop(columns=['노면상태'], inplace=True)

#기상상태 범주화
df['기상상태_위험도'] = df['기상상태'].apply(lambda x: '매우 위험(안개, 눈)' if x in ['안개', '눈'] else ('위험(비, 흐림)' if x in ['비', '흐림'] else x))
df.drop(columns=['기상상태'], inplace=True)

# 가해운전자 차종 범주화
large_vehicle_types = ['건설기계', '승용', '승합', '특수', '화물', '농기계']
small_vehicle_types = ['개인형이동수단(PM)', '기타불명', '사륜오토바이(ATV)', '원동기', '이륜', '자전거']

def classify_driver_vehicle_type(vehicle_type):
    if vehicle_type in large_vehicle_types:
        return '중대형'
    elif vehicle_type in small_vehicle_types:
        return '소형'
    else:
        return vehicle_type

df['가해운전자 차종_분류'] = df['가해운전자 차종'].apply(classify_driver_vehicle_type)

# 피해운전자 차종 범주화
def classify_victim_vehicle_type(vehicle_type):
    if vehicle_type in large_vehicle_types:
        return '중대형'
    elif vehicle_type in small_vehicle_types:
        return '소형'
    else:
        return vehicle_type

df['피해운전자 차종_분류'] = df['피해운전자 차종'].apply(classify_victim_vehicle_type)
df.drop(columns=['가해운전자 차종', '피해운전자 차종'], inplace=True)




In [88]:
# 인코딩
categorical_features = ['사고유형', '법규위반', '노면상태_위험도', '기상상태_위험도', '도로형태',
                        '가해운전자 차종_분류', '가해운전자 성별', '피해운전자 차종_분류',
                        '피해운전자 성별', '평일/주말', '시간대',
                        '가해운전자 연령대', '피해운전자 연령대','가해운전자 상해정도', '피해운전자 상해정도']

df = pd.get_dummies(df, columns=categorical_features)


In [ ]:
# 이상치 제거
z_scores = np.abs(stats.zscore(df['ECLO']))
outliers = (z_scores > 3)
df_no_outliers = df[~outliers]

# MINMAX 스케일러 적용

minmax_scaler = MinMaxScaler()
eclo_minmax_scaled = minmax_scaler.fit_transform(df_no_outliers['ECLO'].values.reshape(-1, 1))
df_no_outliers['ECLO_scaled'] = eclo_minmax_scaled

#데이터 분할

In [90]:
# 피처 선택
features = ['사고유형_차대사람 - 기타', '사고유형_차대사람 - 길가장자리구역통행중', '사고유형_차대사람 - 보도통행중', '사고유형_차대사람 - 차도통행중', '사고유형_차대사람 - 횡단중', '사고유형_차대차 - 기타', '사고유형_차대차 - 정면충돌', '사고유형_차대차 - 추돌', '사고유형_차대차 - 측면충돌', '사고유형_차대차 - 후진중충돌', '사고유형_차량단독 - 공작물충돌', '사고유형_차량단독 - 기타', '사고유형_차량단독 - 도로외이탈 - 기타', '사고유형_차량단독 - 도로외이탈 - 추락', '사고유형_차량단독 - 전도전복 - 전도', '사고유형_차량단독 - 전도전복 - 전복', '사고유형_차량단독 - 주/정차차량 충돌', '법규위반_과속', '법규위반_교차로운행방법위반', '법규위반_기타', '법규위반_보행자보호의무위반', '법규위반_불법유턴', '법규위반_신호위반', '법규위반_안전거리미확보', '법규위반_안전운전불이행', '법규위반_중앙선침범', '법규위반_직진우회전진행방해', '법규위반_차로위반', '노면상태_위험도_건조', '노면상태_위험도_위험(적설, 침수)', '노면상태_위험도_젖음/습기', '기상상태_위험도_기타', '기상상태_위험도_맑음', '기상상태_위험도_매우 위험(안개, 눈)', '기상상태_위험도_위험(비, 흐림)', '도로형태_교차로 - 교차로부근', '도로형태_교차로 - 교차로안', '도로형태_교차로 - 교차로횡단보도내', '도로형태_기타 - 기타', '도로형태_단일로 - 고가도로위', '도로형태_단일로 - 교량', '도로형태_단일로 - 기타', '도로형태_단일로 - 지하차도(도로)내', '도로형태_단일로 - 터널', '도로형태_미분류 - 미분류', '도로형태_주차장 - 주차장', '가해운전자 차종_분류_소형', '가해운전자 차종_분류_중대형', '가해운전자 성별_기타불명', '가해운전자 성별_남', '가해운전자 성별_여', '피해운전자 차종_분류_보행자', '피해운전자 차종_분류_소형', '피해운전자 차종_분류_중대형', '피해운전자 성별_기타불명', '피해운전자 성별_남', '피해운전자 성별_여', '평일/주말_주말', '평일/주말_평일', '시간대_야간', '시간대_주간', '가해운전자 연령대_노년층', '가해운전자 연령대_미성년층', '가해운전자 연령대_중장년층', '가해운전자 연령대_청년층', '피해운전자 연령대_노년층', '피해운전자 연령대_미성년층', '피해운전자 연령대_중장년층', '피해운전자 연령대_청년층', '가해운전자 상해정도_경상', '가해운전자 상해정도_기타불명', '가해운전자 상해정도_부상신고', '가해운전자 상해정도_사망', '가해운전자 상해정도_상해없음', '가해운전자 상해정도_중상', '피해운전자 상해정도_경상', '피해운전자 상해정도_기타불명', '피해운전자 상해정도_부상신고', '피해운전자 상해정도_사망', '피해운전자 상해정도_상해없음', '피해운전자 상해정도_중상']

In [91]:
# 훈련 데이터 및 테스트 데이터 분할
X = df_no_outliers[features]
y = df_no_outliers['ECLO_scaled']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#모델 훈련

In [ ]:
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

# 모델 훈련
xgb_regressor = XGBRegressor(random_state=42)
xgb_regressor.fit(X_train, y_train)

# 모델 예측
y_pred_train_xgb = xgb_regressor.predict(X_train)
y_pred_test_xgb = xgb_regressor.predict(X_test)

def evaluate_model_performance(model_name, y_train, y_pred_train, y_test, y_pred_test):
    print("\n{} 성능 평가:".format(model_name))
    print("훈련 세트 R2 점수:", r2_score(y_train, y_pred_train))
    print("테스트 세트 R2 점수:", r2_score(y_test, y_pred_test))
    print("훈련 세트 MSE:", mean_squared_error(y_train, y_pred_train))
    print("테스트 세트 MSE:", mean_squared_error(y_test, y_pred_test))
    print("훈련 세트 MAE:", mean_absolute_error(y_train, y_pred_train))
    print("테스트 세트 MAE:", mean_absolute_error(y_test, y_pred_test))

# 모델 평가
evaluate_model_performance("XGBoost", y_train, y_pred_train_xgb, y_test, y_pred_test_xgb)


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# XGBRegressor 모델 생성
xgb_regressor = XGBRegressor(random_state=42)

# 탐색할 하이퍼파라미터 공간 정의
param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.5, 0.75, 1],
    'colsample_bytree': [0.5, 0.75, 1],
    'gamma': [0, 0.1, 0.2]
}

# 평가 지표 설정 (R2, MSE, MAE)
scoring = {
    'r2': make_scorer(r2_score),
    'mse': make_scorer(mean_squared_error),
    'mae': make_scorer(mean_absolute_error)
}

# 랜덤 서치 객체 생성
random_search = RandomizedSearchCV(estimator=xgb_regressor,
                                   param_distributions=param_grid,
                                   n_iter=50,
                                   scoring=scoring,
                                   refit='r2',
                                   cv=5,
                                   verbose=1,
                                   random_state=42)

# 랜덤 서치를 통한 하이퍼파라미터 탐색
random_search.fit(X_train, y_train)

# 최적 모델
best_xgb_model = random_search.best_estimator_

# 테스트 세트 예측
y_pred_train_xgb = best_xgb_model.predict(X_train)
y_pred_test_xgb = best_xgb_model.predict(X_test)

# 모델 평가
evaluate_model_performance("Optimized XGBoost", y_train, y_pred_train_xgb, y_test, y_pred_test_xgb)

# 최적 파라미터 출력
print("\n최적 파라미터:", random_search.best_params_)


In [70]:
import shap

# SHAP 값을 계산합니다.
explainer = shap.Explainer(xgb_regressor, X_train)
shap_values = explainer.shap_values(X_train)

# 각 특성의 SHAP 값의 평균을 계산합니다.
mean_abs_shap_values = np.mean(np.abs(shap_values), axis=0)

# 특성 이름과 평균 SHAP 값을 매핑합니다.
feature_names = X_train.columns.tolist()
feature_importance = dict(zip(feature_names, mean_abs_shap_values))

# 중요도를 내림차순으로 정렬합니다.
sorted_feature_importance = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)

# 중요도를 텍스트로 출력합니다.
print("XGBoost 모델의 특성별 SHAP 중요도:")
for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance:.4f}")


100%|===================| 80729/81090 [04:08<00:01]       

XGBoost 모델의 특성별 SHAP 중요도:
피해운전자 상해정도_중상: 0.0519
피해운전자 차종_분류_중대형: 0.0430
가해운전자 상해정도_경상: 0.0274
피해운전자 상해정도_경상: 0.0242
가해운전자 상해정도_중상: 0.0110
평일/주말_주말: 0.0093
가해운전자 차종_분류_소형: 0.0090
사고유형_차대차 - 추돌: 0.0074
가해운전자 상해정도_사망: 0.0064
시간대_야간: 0.0060
피해운전자 상해정도_부상신고: 0.0056
가해운전자 상해정도_부상신고: 0.0036
법규위반_보행자보호의무위반: 0.0032
법규위반_신호위반: 0.0032
가해운전자 상해정도_상해없음: 0.0029
피해운전자 상해정도_상해없음: 0.0026
피해운전자 상해정도_사망: 0.0021
피해운전자 연령대_청년층: 0.0019
피해운전자 차종_분류_보행자: 0.0019
법규위반_안전운전불이행: 0.0018
가해운전자 상해정도_기타불명: 0.0018
피해운전자 연령대_미성년층: 0.0016
피해운전자 연령대_중장년층: 0.0014
도로형태_교차로 - 교차로안: 0.0013
법규위반_중앙선침범: 0.0013
가해운전자 연령대_청년층: 0.0013
사고유형_차대차 - 측면충돌: 0.0012
도로형태_단일로 - 기타: 0.0010
피해운전자 성별_여: 0.0010
가해운전자 성별_남: 0.0010
피해운전자 연령대_노년층: 0.0009
법규위반_안전거리미확보: 0.0009
가해운전자 연령대_중장년층: 0.0009
가해운전자 연령대_노년층: 0.0008
사고유형_차량단독 - 기타: 0.0008
사고유형_차대차 - 정면충돌: 0.0007
가해운전자 연령대_미성년층: 0.0007
피해운전자 차종_분류_소형: 0.0007
피해운전자 성별_남: 0.0006
도로형태_교차로 - 교차로부근: 0.0006
사고유형_차대차 - 기타: 0.0006
법규위반_교차로운행방법위반: 0.0005
기상상태_위험도_맑음: 0.0005
가해운전자 성별_여: 0.0005
사고유형_차대차 

#SUB

In [ ]:
print(df.columns.tolist())

In [ ]:
print(df.head())

In [14]:
# 파일로 훈련결과 출력
result_df = pd.DataFrame({
    'Actual': y_test,
    'Gradient Boosting Predicted': y_pred_test_gb
})

# CSV 파일로 저장
result_df.to_csv('/content/drive/MyDrive/prediction_results.csv', index=False)


In [83]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/sample_data.csv', index=False)

#시각화 관련

데이터탐색에 들어갈 자료 생성

In [ ]:
# 빈도표
plt.figure(figsize=(16, 12))
sns.countplot(x='피해운전자 연령', data=df, palette='viridis')
plt.ylabel('빈도')
plt.show()


In [ ]:
# 커널 밀도 추정 그래프
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df, x='사고발생시간', fill=True, color='skyblue')
plt.title('사고발생시간별 분포')
plt.xlabel('사고발생시간')
plt.ylabel('밀도')
plt.show()


시각화에 들어갈 자료 생성

In [ ]:
# 사고내용별 ECLO
avg_eclo_by_accident = df.groupby('시군구')['ECLO'].mean().reset_index()
avg_eclo_all = df['ECLO'].mean()

# 막대 그래프
plt.figure(figsize=(12, 8))
sns.barplot(x='시군구', y='ECLO', data=avg_eclo_by_accident, palette='viridis')
plt.ylabel('평균 ECLO')
plt.axhline(avg_eclo_all, color='r', linestyle='--', label='전체 평균 ECLO')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
def categorize_age(age):
    if age <= 19:
        return '0~19'
    elif age <= 45:
        return '20~45'
    elif age <= 60:
        return '46~60'
    else:
        return '61 이상'

# 연령을 범주
df['피해운전자 연령대'] = df['피해운전자 연령'].apply(categorize_age)

# 사고내용별 ECLO
avg_eclo_by_accident = df.groupby('피해운전자 연령대')['ECLO'].mean().reset_index()
avg_eclo_all = df['ECLO'].mean()

# 막대 그래프
plt.figure(figsize=(12, 8))
sns.barplot(x='피해운전자 연령대', y='ECLO', data=avg_eclo_by_accident, palette='viridis')
plt.ylabel('평균 ECLO')
plt.axhline(avg_eclo_all, color='r', linestyle='--', label='전체 평균 ECLO')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 3가지 이상의 피처를 활용한 히트맵

heatmap_data = df.pivot_table(index='사고발생시간', columns='법규위반', values='ECLO', aggfunc='mean')

plt.figure(figsize=(14, 10))
sns.heatmap(heatmap_data, cmap='viridis', annot=True, fmt=".2f", linewidths=.5)
plt.title('사고 발생시간과 법규위반에 따른 ECLO 평균')
plt.xlabel('법규위반')
plt.ylabel('사고발생시간')
plt.show()
